In [1]:
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 220 kB 8.7 MB/s 


In [2]:
!pip install pycryptodome

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.3 MB 7.8 MB/s 


In [3]:
import PyPDF2
import re
import pandas as pd

In [4]:
pdfReader = PyPDF2.PdfFileReader('221006_Significant_Cyber_Incidents.pdf')

In [5]:
print(pdfReader.numPages)

75


Extract text info to dataframe

In [37]:
month_match = r'\n[A-Z][a-z]+ 20[012][0-9].'
data = None

for i in range(pdfReader.numPages):
  contents = pdfReader.getPage(i).extract_text()

  # remove header
  contents = re.sub('\n \nCenter for Strategic and International Studies \(CSIS\) \| Washington, D.C. ', '', contents)

  if i == 0:
    contents = re.sub('      \n \n \n \n \n \nSignificant Cyber Incidents Since 2006  \n \nThis list is a work in progress that we update as new incidents come to light. If you have \nsuggestions for additions, se nd them to strategictech @csis.org.  Significance is in the eye of the \nbeholder, but we focus on cyber -attacks on government agencies, defense and high tech \ncompanies, or economic crimes with losses of more than a million dollars.  \n \n', '', contents)


  mth_start_idx = re.finditer(month_match, contents)

  start = 0

  # iterate through each entry
  for j in mth_start_idx:
    # get entry text
    entry = re.sub('\n', '', contents[start:j.start(0)])
    entry = re.sub('\s\s+', ' ', entry)

    # check for trailing descriptions
    if start == 0 and j.start(0) > 0 and data is not None:
      data.iloc[-1].Description = ' '.join([data.iloc[-1].Description, entry])
      start = j.start(0)
      continue

    if data is not None:
      # add entry to dataframe
      entry = pd.DataFrame({'Description': [entry] })
      data = data.append(entry, ignore_index=True)
      start = j.start(0)
    else:
      data = pd.DataFrame({'Description': [entry] })
    
  # add last entry
  entry = re.sub('\n', '', contents[start:])
  entry = re.sub('\s\s+', ' ', entry)
  entry = pd.DataFrame({'Description': [entry] })
  data = data.append(entry, ignore_index=True)

In [38]:
data

,Description
0,September 2022. Iranian hackers targeted Alban...
1,September 2022. Hackers targeted the state -le...
2,September 2022. China accused the U.S. Nationa...
3,September 2022. The group Anonymous took respo...
4,September 2022. Hackers targeted the Mexican D...
...,...
861,December 2006. NASA was forced to block emails...
862,November 2006. Hackers attempted to penetrate ...
863,August 2006. A senior A ir Force Officer state...
864,May 2006. The Department of State’s networks w...


In [39]:
data.to_csv('cybersecurity_incidents_descriptions.csv')